In [2]:
from data_loader import QAGNN_RawDataLoader
import torch
import json


cfg = json.load(fp=open('config.json', 'r'))


db = QAGNN_RawDataLoader(cfg['cp_emb_path'], cfg['train_stmt_path'], cfg['train_adj_path'],
                         cfg['dev_stmt_path'], cfg['dev_adj_path'],
                         cfg['test_stmt_path'], cfg['test_adj_path'],
                         batch_size=cfg['batch_size'], lm_name=cfg['lm_name'],
                         n_ntype=cfg['n_ntype'], n_etype=cfg['n_etype'],
                         max_node_num=cfg['max_n_nodes'], max_seq_length=cfg['max_seq_len'])

cp_names = open('data/cpnet/concept.txt', 'r').read().splitlines()
cp_names

['ab_extra',
 'ab_intra',
 'abactinal',
 'actinal',
 'abandon',
 'acquire',
 'arrogate',
 'embrace',
 'engage',
 'gain',
 'join',
 'maintain',
 'retain',
 'unite',
 'abandonment',
 'acquisition',
 'abapical',
 'apical',
 'abase',
 'exalt',
 'extoll',
 'abash',
 'embolden',
 'reassure',
 'abate',
 'augment',
 'abaxial',
 'adaxial',
 'abbreviate',
 'lengthen',
 'abderian',
 'agelastic',
 'abducent',
 'adducent',
 'abduction',
 'adduction',
 'abductive',
 'deduce',
 'abductor',
 'abductee',
 'adductor',
 'abideable',
 'insupportable',
 'intolerable',
 'unabideable',
 'unbearable',
 'abience',
 'adience',
 'abient',
 'adient',
 'ability',
 'inability',
 'abiogenesis',
 'biogenesis',
 'transformism',
 'abjectly',
 'proudly',
 'abjugate',
 'adjugate',
 'able',
 'cane',
 'cannot',
 'disability',
 'unable',
 'abled',
 'disabled',
 'abluminal',
 'adluminal',
 'abnormal',
 'normal',
 'abnormality',
 'normality',
 'abolish',
 'establish',
 'abolition',
 'establishment',
 'aborad',
 'orad',
 'abor

In [3]:
qad = db.test_qad



100%|██████████| 1140/1140 [00:03<00:00, 325.88it/s]


| ori_adj_len: mu 119.20 sigma 93.49 | adj_len: 106.22 | prune_rate： 0.16 | qc_num: 7.38 | ac_num: 2.05 |


In [4]:
i = 0
qid = qad.qids[i]
cp_ids = qad.node_concept_ids[i]
cp_ids = cp_ids[1:] - 1

# print([cp_names[j] for j in cp_ids])

In [5]:
import json
from pprint import pprint

f = open('data/csqa/statement/test.statement.jsonl', 'r')
stmts = [json.loads(jline) for jline in f.read().splitlines()]
stmt = [s for s in stmts if s['id'] == qid][0]
# pprint(stmt)

In [6]:
from transformers import AutoModel
lm = 'roberta-large'
encoder = AutoModel.from_pretrained(lm, output_hidden_states=True)
sent_dim = encoder.config.hidden_size
print(sent_dim)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


1024


In [12]:
input_ids, attention_mask, token_type_ids, output_mask = [x[0:1] for x in qad.text_data]
# print(input_ids)
outputs = encoder(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)

all_hidden_states = outputs[-1]
hidden_states = all_hidden_states[-1]

print(hidden_states.size())
hidden_states

torch.Size([1, 128, 1024])


tensor([[[ 0.0329,  0.0337,  0.0040,  ..., -0.0658,  0.0767,  0.0680],
         [ 0.2004,  0.0547, -0.9675,  ..., -0.0487, -0.0216,  0.0416],
         [ 0.2271,  0.1094, -0.1333,  ...,  0.6103, -0.1497,  0.0443],
         ...,
         [ 0.0329,  0.0337,  0.0040,  ..., -0.0658,  0.0767,  0.0680],
         [ 0.0329,  0.0337,  0.0040,  ..., -0.0658,  0.0767,  0.0680],
         [ 0.0329,  0.0337,  0.0040,  ..., -0.0658,  0.0767,  0.0680]]],
       grad_fn=<NativeLayerNormBackward0>)

In [13]:
print(input_ids)

tensor([[    0,   133,  1139,  3138,    21,    10,   543,  1331,    13,     5,
           588, 10321,     6,    24,    21,   235,   220,     7,    10,   239,
          1430,    99,   116,     2,     2, 10551,   709,     2,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [14]:
print(attention_mask)

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])


In [15]:
print(token_type_ids)

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])


In [16]:
print(output_mask)

tensor([[ True, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False,  True,  True, False, False,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  